# Auto-Graph-RAG Modular Architecture Demo

This notebook demonstrates the new modular architecture of Auto-Graph-RAG, showing how each component can be used independently or composed together.

## Key Benefits
- **Independent Testing**: Test each component with known inputs
- **Reusable Assets**: Reuse expensive operations (schemas, datasets, models)
- **Flexible Composition**: Mix and match modules or skip steps
- **Better Debugging**: Isolate issues to specific components
- **Incremental Development**: Work on parts without full pipeline

## Setup and Imports

In [1]:
import os
import json
import networkx as nx
from pathlib import Path
import tempfile
import shutil
from dotenv import load_dotenv
load_dotenv()

# Import modular components
from auto_graph_rag.modules import (
    GraphBuilder,
    GraphExplorer, 
    DataGenerator,
    ModelTrainer,
    QueryExecutor
)

# Original interface for comparison
from auto_graph_rag import GraphRAG

print("✅ Imports successful!")
print("Available modules:")
for module in [GraphBuilder, GraphExplorer, DataGenerator, ModelTrainer, QueryExecutor]:
    print(f"  - {module.__name__}")

✅ Imports successful!
Available modules:
  - GraphBuilder
  - GraphExplorer
  - DataGenerator
  - ModelTrainer
  - QueryExecutor


## Environment Setup

Let's set up our working directory and check for required API keys.

In [2]:
# Create working directory
work_dir = Path("./modular_demo_workspace")
work_dir.mkdir(exist_ok=True)

print(f"📁 Working directory: {work_dir.absolute()}")

# Check environment - require API key for full demo
has_openai = bool(os.getenv("OPENAI_API_KEY"))
has_hf = bool(os.getenv("HF_TOKEN"))

print(f"🔑 OpenAI API Key: {'✅ Available' if has_openai else '❌ Missing'}")
print(f"🤗 HuggingFace Token: {'✅ Available' if has_hf else '❌ Missing (optional for some models)'}")

if not has_openai:
    print("\n❌ Error: OPENAI_API_KEY is required for this demo")
    print("Please set it with: os.environ['OPENAI_API_KEY'] = 'your-key-here'")
    raise ValueError("OPENAI_API_KEY environment variable is required")

📁 Working directory: /Users/benjaminfriedman/repos/auto-graph-rag/modular_demo_workspace
🔑 OpenAI API Key: ✅ Available
🤗 HuggingFace Token: ✅ Available


## Step 1: Graph Builder Module

The `GraphBuilder` creates Kuzu databases from NetworkX graphs or raw data. This can be used standalone without any other components.

In [3]:
# Create a sample company graph
def create_sample_graph():
    """Create a sample company graph for demonstration."""
    G = nx.DiGraph()
    
    # Add employees
    employees = [
        ("emp1", {"name": "Alice Johnson", "department": "Engineering", "salary": 120000, "level": "Senior"}),
        ("emp2", {"name": "Bob Smith", "department": "Engineering", "salary": 95000, "level": "Junior"}),
        ("emp3", {"name": "Carol White", "department": "Marketing", "salary": 85000, "level": "Mid"}),
        ("emp4", {"name": "David Brown", "department": "Sales", "salary": 90000, "level": "Senior"}),
    ]
    
    for emp_id, attrs in employees:
        G.add_node(emp_id, type="Employee", **attrs)
    
    # Add departments
    departments = [
        ("dept1", {"name": "Engineering", "budget": 2000000, "head_count": 25}),
        ("dept2", {"name": "Marketing", "budget": 800000, "head_count": 10}),
        ("dept3", {"name": "Sales", "budget": 1200000, "head_count": 15}),
    ]
    
    for dept_id, attrs in departments:
        G.add_node(dept_id, type="Department", **attrs)
    
    # Add projects
    projects = [
        ("proj1", {"name": "Alpha", "budget": 500000, "status": "Active"}),
        ("proj2", {"name": "Beta", "budget": 300000, "status": "Planning"}),
    ]
    for proj_id, attrs in projects:
        G.add_node(proj_id, type="Project", **attrs)
    
    # Add relationships
    G.add_edge("emp1", "dept1", type="BELONGS_TO", since="2020-01-15")
    G.add_edge("emp2", "dept1", type="BELONGS_TO", since="2023-03-01")
    G.add_edge("emp3", "dept2", type="BELONGS_TO", since="2021-06-15")
    G.add_edge("emp4", "dept3", type="BELONGS_TO", since="2019-09-01")
    
    G.add_edge("emp1", "proj1", type="WORKED_ON", hours=320, role="Lead")
    G.add_edge("emp2", "proj1", type="WORKED_ON", hours=480, role="Developer")
    G.add_edge("emp3", "proj2", type="WORKED_ON", hours=200, role="Marketing Lead")
    
    G.add_edge("proj1", "dept1", type="OWNED_BY")
    G.add_edge("proj2", "dept2", type="OWNED_BY")
    
    return G

# Create the graph
sample_graph = create_sample_graph()

print(f"📊 Created sample graph:")
print(f"  - Nodes: {sample_graph.number_of_nodes()}")
print(f"  - Edges: {sample_graph.number_of_edges()}")
print(f"  - Node types: {set(data['type'] for _, data in sample_graph.nodes(data=True))}")
print(f"  - Edge types: {set(data['type'] for _, _, data in sample_graph.edges(data=True))}")

📊 Created sample graph:
  - Nodes: 9
  - Edges: 9
  - Node types: {'Project', 'Department', 'Employee'}
  - Edge types: {'BELONGS_TO', 'WORKED_ON', 'OWNED_BY'}


In [4]:
# Initialize GraphBuilder and build database
builder = GraphBuilder()

# Show module info
print("🔧 GraphBuilder Info:")
info = builder.get_info()
for key, value in info.items():
    print(f"  {key}: {value}")

print("\n📦 Building graph database...")

# Extract labels
node_labels = {node: data["type"] for node, data in sample_graph.nodes(data=True)}
edge_labels = {(u, v): data["type"] for u, v, data in sample_graph.edges(data=True)}

# Build the database
db_path = work_dir / "company_db"
stats = builder.build_from_networkx(
    graph=sample_graph,
    db_path=db_path,
    graph_name="company",
    node_labels=node_labels,
    edge_labels=edge_labels
)

print("\n✅ Graph database created!")
print(f"📊 Statistics:")
for key, value in stats.items():
    print(f"  {key}: {value}")

🔧 GraphBuilder Info:
  name: GraphBuilder
  version: 1.0.0
  description: Build Kuzu graphs from NetworkX or raw data
  inputs: {'networkx': ['graph', 'db_path', 'graph_name', 'node_labels', 'edge_labels'], 'raw_data': ['nodes', 'edges', 'db_path', 'graph_name']}
  outputs: ['statistics', 'db_path']

📦 Building graph database...

✅ Graph database created!
📊 Statistics:
  node_tables: ['company_Employee', 'company_Department', 'company_Project']
  edge_tables: ['company_BELONGS_TO_Employee_to_Department', 'company_WORKED_ON_Employee_to_Project', 'company_OWNED_BY_Project_to_Department']
  total_nodes: 9
  total_edges: 9
  db_path: modular_demo_workspace/company_db
  graph_name: company


## Step 2: Graph Explorer Module

The `GraphExplorer` analyzes an existing Kuzu database to understand its schema using LLM analysis. It can work with any Kuzu database, regardless of how it was created.

In [6]:
# Initialize GraphExplorer 
explorer = GraphExplorer(llm_provider="openai", llm_model="gpt-4")

# Show module info
print("🔧 GraphExplorer Info:")
info = explorer.get_info()
for key, value in info.items():
    print(f"  {key}: {value}")

print("\n🕵️ Exploring graph schema...")

# Explore the database we just created
schema_path = work_dir / "company_schema.json"
schema = explorer.explore_from_db(
    db_path=db_path,
    max_samples=15,
    save_schema_to=schema_path
)

print("\n✅ Schema exploration complete!")

# Display schema results
print(f"\n📋 Discovered Schema:")
print(f"  Summary: {schema['summary']}")
print(f"  Node Types: {list(schema['nodes'].keys())}")
print(f"  Edge Types: {list(schema['edges'].keys())}")

# Show detailed info for one node type
if 'Employee' in schema['nodes']:
    emp_info = schema['nodes']['Employee']
    print(f"\n👤 Employee Node Details:")
    print(f"  Description: {emp_info['description']}")
    print(f"  Properties: {emp_info['properties']}")
    print(f"  Example: {emp_info['example_values']}")

🔧 GraphExplorer Info:
  name: GraphExplorer
  version: 1.0.0
  description: Explore and understand graph schemas using LLM
  inputs: {'from_db': ['db_path', 'max_samples', 'save_schema_to'], 'from_adapter': ['kuzu_adapter', 'max_samples', 'save_schema_to']}
  outputs: ['schema', 'summary', 'node_types', 'edge_types']

🕵️ Exploring graph schema...
DEBUG: DB schema has 3 node tables, 3 edge tables
DEBUG: Sampled 6 table groups
DEBUG: Total sample items: 18
DEBUG: Schema conversion - nodes: ['company_Employee', 'company_Department', 'company_Project']
DEBUG: Schema conversion - edges: ['company_WORKED_ON_Employee_to_Project', 'company_BELONGS_TO_Employee_to_Department', 'company_OWNED_BY_Project_to_Department']

✅ Schema exploration complete!

📋 Discovered Schema:
  Summary: This graph represents the structure of a company, including its employees, departments, and projects. It captures the relationships between these entities, such as which employees work on which projects, which employe

## Step 3: Data Generator Module

The `DataGenerator` creates training datasets from graph schemas. It can work with any schema JSON file, regardless of how it was created.

In [7]:
# Initialize DataGenerator
generator = DataGenerator(llm_provider="openai", llm_model="gpt-4")
schema_path = work_dir / "company_schema.json"# Show module info
db_path = work_dir / "company_db"


print("🔧 DataGenerator Info:")
info = generator.get_info()
for key, value in info.items():
    print(f"  {key}: {value}")

print("\n📝 Generating training data...")

# Generate dataset from the schema we created
dataset_path = work_dir / "company_dataset.jsonl"
dataset = generator.generate_from_schema(
    schema_path=schema_path,
    num_examples=100,  # Small number for demo
    output_path=dataset_path,
    complexity_distribution={
        1: 0.3,  # Simple lookups
        2: 0.3,  # Filtered queries  
        3: 0.2,  # Relationships
        4: 0.2,  # Aggregations
    },
    db_path=db_path  # For validation
)

print("\n✅ Training data generated!")

# Show dataset statistics
print(f"\n📊 Dataset Statistics:")
print(f"  Total examples: {len(dataset)}")

# Analyze complexity distribution
complexity_counts = {}
intent_counts = {}
for item in dataset:
    complexity = item.get('complexity', 0)
    intent = item.get('intent', 'unknown')
    complexity_counts[complexity] = complexity_counts.get(complexity, 0) + 1
    intent_counts[intent] = intent_counts.get(intent, 0) + 1

print(f"  Complexity distribution: {complexity_counts}")
print(f"  Intent distribution: {intent_counts}")

# Show sample examples
print(f"\n💡 Sample Training Examples:")
for i, example in enumerate(dataset[:3], 1):
    print(f"\n  Example {i}:")
    print(f"    Question: {example.get('question', 'N/A')}")
    print(f"    Cypher: {example.get('cypher', 'N/A')[:80]}...")
    print(f"    Complexity: {example.get('complexity', 'N/A')}")
    print(f"    Intent: {example.get('intent', 'N/A')}")

🔧 DataGenerator Info:
  name: DataGenerator
  version: 1.0.0
  description: Generate training data from graph schemas
  inputs: {'from_schema': ['schema_path', 'num_examples', 'output_path', 'complexity_distribution', 'db_path'], 'from_dict': ['schema', 'num_examples', 'output_path', 'complexity_distribution', 'kuzu_adapter']}
  outputs: ['dataset', 'statistics']

📝 Generating training data...
DEBUG: Generating 30 examples for complexity 1
DEBUG: Generating batch of 20, remaining=30, attempts=0
DEBUG: LLM prompt will use node_types: ['company_Employee', 'company_Department', 'company_Project']
DEBUG: LLM prompt will use edge_types: ['company_WORKED_ON_Employee_to_Project', 'company_BELONGS_TO_Employee_to_Department', 'company_OWNED_BY_Project_to_Department']
DEBUG: LLM prompt (first 500 chars): Graph Schema:
{
  "nodes": {
    "company_Employee": {
      "properties": [
        "id",
        "salary",
        "name",
        "department",
        "level"
      ],
      "description": "

## Step 4: Model Trainer Module

The `ModelTrainer` fine-tunes language models on training datasets. It can work with any dataset file, regardless of how it was created.

**Note**: Model training is computationally expensive and time-consuming. In this demo, we'll show the setup but skip the actual training.

In [ ]:
# Initialize ModelTrainer
trainer = ModelTrainer()


# Show module info
print("🔧 ModelTrainer Info:")
info = trainer.get_info()
for key, value in info.items():
    print(f"  {key}: {value}")

# Show training configuration
print("\n⚙️ Training Configuration:")
dataset_path = work_dir / "company_dataset.jsonl"
training_config = {
    "dataset_path":dataset_path,
    "base_model": "meta-llama/Llama-3.2-1B-Instruct",
    "epochs": 5,
    "learning_rate": 5e-4,
    "batch_size": 1,
    "lora_rank": 8,
    "output_dir": str(work_dir / "company_model")
}

for key, value in training_config.items():
    print(f"  {key}: {value}")

print("\n🎯 Model Training:")

model = trainer.train_from_file(
    dataset_path=Path(training_config['dataset_path']),
    model_name=training_config['base_model'],
    output_dir=Path(training_config['output_dir']),
    epochs=training_config['epochs'],
    learning_rate=training_config['learning_rate'],
    batch_size=training_config['batch_size'],
    lora_rank=training_config['lora_rank']
)



🔧 ModelTrainer Info:
  name: ModelTrainer
  version: 1.0.0
  description: Fine-tune models on graph query datasets
  inputs: {'from_file': ['dataset_path', 'model_name', 'output_dir', 'epochs', 'learning_rate', 'batch_size', 'lora_rank'], 'from_data': ['dataset', 'model_name', 'output_dir', 'epochs', 'learning_rate', 'batch_size', 'lora_rank']}
  outputs: ['model', 'training_stats', 'model_path']

⚙️ Training Configuration:
  dataset_path: modular_demo_workspace/company_dataset.jsonl
  base_model: meta-llama/Llama-3.2-1B-Instruct
  epochs: 5
  learning_rate: 0.0005
  batch_size: 1
  lora_rank: 8
  output_dir: modular_demo_workspace/company_model

🎯 Model Training:
trainable params: 5,636,096 || all params: 1,241,450,496 || trainable%: 0.4540


Map:   0%|          | 0/90 [00:00<?, ? examples/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

/Users/benjaminfriedman/repos/auto-graph-rag/.venv/lib/python3.11/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


## Step 5: Query Executor Module

The `QueryExecutor` executes natural language queries using fine-tuned models and graph databases. It can work with any trained model and database combination.

**This step uses the actual trained model from Step 4 and the database from Step 1, demonstrating how assets can be reused across modules.**

In [5]:
# Initialize QueryExecutor
executor = QueryExecutor()

# Use the trained model from Step 4 and database from Step 1
model_path = work_dir / "company_model"  # From Step 4 ModelTrainer
db_path = work_dir / "company_db"        # From Step 1 GraphBuilder

print("🔧 QueryExecutor Info:")
info = executor.get_info()
for key, value in info.items():
    print(f"  {key}: {value}")

print(f"\n🎯 Using Assets from Previous Steps:")
print(f"  Model: {model_path} (from Step 4)")
print(f"  Database: {db_path} (from Step 1)")

# Test questions we'd like to ask
test_questions = [
    "Who are all the employees?",
    "Which employees work in Engineering?", 
    "What projects has Alice Johnson worked on?",
    "What is the average salary by department?",
    "Which departments have the highest budgets?"
]

print("\n❓ Test Questions:")
for i, question in enumerate(test_questions, 1):
    print(f"  {i}. {question}")

print("\n🤖 Query Execution with Trained Model:")   
for i, question in enumerate(test_questions, 1):
    print(f"\n  Query {i}: {question}")
    
    result = executor.query_with_model(
        question=question,
        model_path=Path(model_path),
        db_path=Path(db_path),
        return_cypher=True,
        format_results=True
    )
    
    if result['success']:
        print(f"    ✅ Generated Cypher: {result['cypher']}")
        print(f"    📊 Results: {result['count']} rows")
        if 'results' in result and result['results']:
            # Show first few results
            for j, row in enumerate(result['results'][:2]):
                print(f"      Row {j+1}: {row}")
            if len(result['results']) > 2:
                print(f"      ... and {len(result['results']) - 2} more rows")
    else:
        print(f"    ❌ Error: {result['error']}")

Could not detect model name, using default


🔧 QueryExecutor Info:
  name: QueryExecutor
  version: 1.0.0
  description: Execute queries with fine-tuned models
  inputs: {'with_paths': ['question', 'model_path', 'db_path', 'return_cypher', 'format_results'], 'with_instances': ['question', 'model', 'kuzu_adapter', 'return_cypher', 'format_results']}
  outputs: ['success', 'cypher', 'results', 'count', 'error']

🎯 Using Assets from Previous Steps:
  Model: modular_demo_workspace/company_model (from Step 4)
  Database: modular_demo_workspace/company_db (from Step 1)

❓ Test Questions:
  1. Who are all the employees?
  2. Which employees work in Engineering?
  3. What projects has Alice Johnson worked on?
  4. What is the average salary by department?
  5. Which departments have the highest budgets?

🤖 Query Execution with Trained Model:

  Query 1: Who are all the employees?


`torch_dtype` is deprecated! Use `dtype` instead!
/Users/benjaminfriedman/repos/auto-graph-rag/.venv/lib/python3.11/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


Query failed (attempt 1): Binder exception: Table company_OWNED_BY_Employee_to_Employee does not exist.
Query failed (attempt 2): Binder exception: Table company_OWNED_BY_Employee_to_Employee does not exist.
Could not detect model name, using default


...

    ❌ Error: Binder exception: Function LIST_EXTRACT did not receive correct arguments:
Actual:   (NODE,STRING)
Expected: (LIST,INT64) -> ANY
          (STRING,INT64) -> STRING
          (ARRAY,INT64) -> ANY



  Query 3: What projects has Alice Johnson worked on?


Could not detect model name, using default


    ✅ Generated Cypher: MATCH (e:company_Employee {name: 'Alice Johnson'})<-[:company_WORKED_ON_Employee_to_Project]-(p:company_Project) RETURN p.name
    📊 Results: 0 rows

  Query 4: What is the average salary by department?


Query failed (attempt 1): Parser exception: Invalid input < BY>: expected rule ku_Statements (line: 1, offset: 136)
"MATCH (e:company_Employee)-[r:company_BELONGS_TO_Employee_to_Department]->(d:company_Department) RETURN AVG(r.salary) AS Average_Salary BY d.name ORDER BY Average_Salary DESC"
                                                                                                                                         ^^
Query failed (attempt 2): Parser exception: Invalid input < BY>: expected rule ku_Statements (line: 1, offset: 136)
"MATCH (e:company_Employee)-[r:company_BELONGS_TO_Employee_to_Department]->(d:company_Department) RETURN AVG(r.salary) AS Average_Salary BY d.name ORDER BY Average_Salary DESC"
                                                                                                                                         ^^
Could not detect model name, using default


    ❌ Error: Parser exception: Invalid input < BY>: expected rule ku_Statements (line: 1, offset: 136)
"MATCH (e:company_Employee)-[r:company_BELONGS_TO_Employee_to_Department]->(d:company_Department) RETURN AVG(r.salary) AS Average_Salary BY d.name ORDER BY Average_Salary DESC"
                                                                                                                                         ^^

  Query 5: Which departments have the highest budgets?
    ✅ Generated Cypher: MATCH (d:company_Department)-[:company_BELONGS_TO_Employee_to_Department]->(e:company_Department) RETURN e.name AS Department, d.budget AS Budget ORDER BY Budget DESC LIMIT 10
    📊 Results: 0 rows


In [7]:
executor.query_with_model(
        question=question,
        model_path=Path(model_path),
        db_path=Path(db_path),
        return_cypher=True,
        format_results=True
    )

Could not detect model name, using default
Query failed (attempt 1): Binder exception: Cannot find property role for e.


UnboundLocalError: cannot access local variable 're' where it is not associated with a value